In [2]:
import json
import etri

f1 = './koreanframenet/data/bak/test.tsv'
f2 = './koreanframenet/data/bak/training.tsv'
f3 = './koreanframenet/data/bak/training_fe.tsv'
files = [f1, f2, f3]

def getCoNLL(lines, filename):
    filetype = filename.split('/')[-1]
    target_file = './'+filetype
    result = []
    sent = []
    sent_ids = []
    each_sent = {}
    for line in lines:
        line = line.rstrip('\n')
        if line.startswith('#'):
            if line[1] == 's':
                #sent_id = line.split(':')[1]
                each_sent['sent_id'] = line
            else:
                #text = line[6:]
                each_sent['text'] = line
        else:
            if line != '':
                token = line.split('\t')
                sent.append(token)
            else:
                each_sent['conll'] = sent
                result.append(each_sent)
                each_sent = {}
                sent = []
    with open(target_file, 'a') as f:
        for i in result:
            sent_id = i['sent_id']
            text = i['text']
            ori_text = text[6:]
            f.write(sent_id+'\n')
            f.write(text+'\n')
            conll = i['conll']
            etri_result = etri.getETRI_CoNLL2006(ori_text)
            morp = []
            for i in range(len(conll)):
                conll[i][2] = etri_result[i][2]
                if conll[i][11] != '_':
                    conll[i][11] = conll[i][11].lower()
            for token in conll:
                line = '\t'.join(map(str,token))
                f.write(line+'\n')
            f.write('\n')
    print('write:', target_file)

def genData():
    for i in files:
        with open(i, 'r') as f:
            d = f.readlines()
            conll = getCoNLL(d, i)
            
#genData()
        

중국은 파키스탄, 이란, 그리고 북한을 포함한, 여러 나라의 미사일 프로그램에 기술과 전문가를 제공하였다.
이란은 화학 시설과 과거 화학무기 비축에 관한 선언서를 제출하였는데, 이란은 화학무기 금지기구 조사관들이 있는 곳에서 화학무기 생산 장비를 파괴하였고, 화학 산업 시설들에 대해 화학무기 금지기구의 조사를 수 차례 받았다.
이란은 화학 시설과 과거 화학무기 비축에 관한 선언서를 제출하였는데, 이란은 화학무기 금지기구 조사관들이 있는 곳에서 화학무기 생산 장비를 파괴하였고, 화학 산업 시설들에 대해 화학무기 금지기구의 조사를 수 차례 받았다.


In [7]:
from koreanframenet import kfn
def load_kfn():
    with open('./koreanframenet/resource/KFN_lus.json', 'r') as f:
        kolu = json.load(f)
    with open('./koreanframenet/resource/KFN_annotations.json','r') as f:
        koanno = json.load(f)
    print('### load KFN ###')
    return kolu, koanno
kolu, koanno = load_kfn()

### load KFN ###


In [ ]:
# add surfaceform for KNF_lus.json

# get eojeol id (i.e. token id) from 'span' and 'text'
def get_eid(span, ori_text):
    text = ' '.join(ori_text.split())
    sent_list = text.split(' ')
    #print(sent_list)
    b,e = int(span['begin']), int(span['end'])
    n = 0
    k = 0
    ori_text_list = ori_text.split(' ')
    #print(ori_text_list)
    for i in ori_text_list:
        if i == '':
            k = k+1
    #print('k:',k)
    e_num = 0
    e_list = []
    for eojeol in sent_list:
        if eojeol == '':
            pass
        else:
            eojeol_begin_offset = n
            eojeol_end_offset = n+len(eojeol)
            n = eojeol_end_offset+1
            t = (eojeol_begin_offset, eojeol_end_offset, e_num, eojeol)
            e_list.append(t)
            e_num = e_num +1
    begin, end = 0,0
    for i in e_list:
        if b <= i[0]+k or (b >=i[0] and e <= i[1]+k):
            begin = i[2]
            break
    for i in e_list:
        if e <= i[1]+k:
            end = i[2]
            break
    #print(e_list)
    #print(b,e)
    #print(span)
    #print(begin,end)
    return begin, end
def get_eojeol(span, ori_text):
    text = ' '.join(ori_text.split())
    #text = ori_text
    begin, end = get_eid(span, ori_text)
    end = end+1
    text_list = text.split(' ')
    eojeol_list = text_list[begin:end]
    eojeol_list = [ e.replace('.', '') for e in eojeol_list ]
    eojeol_list = [ e.replace(',', '') for e in eojeol_list ]
    eojeol_list = [ e.replace('!', '') for e in eojeol_list ]
    eojeol_list = [ e.replace('?', '') for e in eojeol_list ]
    eojeol = ' '.join(eojeol_list)
    #print(eojeol)
    return eojeol
def add_surfaceforms():
    new_kolu = []
    err = 0
    for i in kolu:
        surface_forms = []
        luid = i['lu_id']

        #if luid == 129:
        #if luid == 193:
        
        #if luid == 5:
        #if luid == 249:
        #if luid == 5392:

        annos = kfn.annotation(luid)
        print('###', luid, i['lu'])
        for anno in annos:
            denos = anno['denotations']
            text = anno['text']
            #print(text)
            for deno in denos:

                if deno['obj'] == 'target' or deno['obj'] == 'Target':
                    #print(deno)
                    target_span = deno['span']
                    b,e = target_span['begin'], target_span['end']
                    #annotation error 제외
                    if type(b) != str:
                        target = get_eojeol(target_span, text)
                        target_anno = text[b:e]
                        if target_anno in target:
                            #print('target_anno:',target_anno, 'target:', target)
                            surface_forms.append(target)
                        else:
                            err = err+1
                            #print('target_anno:',target_anno, 'target:', target)
                    else:
                        #e = e+1
                        pass
        surface_forms = list(set(surface_forms))
        i['surface_forms'] = surface_forms
        #print('')
        new_kolu.append(i)
        print('error:', err)
        #else:
            #pass
        #break
    #print(new_kolu)

    #with open('./resource/KFN_lus.json','w') as f:
        #json.dump(new_kolu, f, ensure_ascii=False, indent=4)
#add_surfaceforms()

In [9]:
def gen_fe_list_for_lu():
    n = 0
    for i in kolu:
        luid = i['lu_id']
        anno = kfn.annotation(luid)
        felist = []
        for j in anno:
            for d in j['denotations']:
                if d['obj'] != 'target':
                    felist.append(d['obj'])
        felist = list(set(felist))
        i['fe_list'] = felist
        print(i)
        break
    print(kolu[0])
        #print(n, '/', len(kolu))
    #with open('./koreanframenet/resource/KFN_lus.json','w') as f:
        #json.dump(kolu, f, ensure_ascii=False, indent=4)
gen_fe_list_for_lu()

{'fid': 43, 'lu': '선언서.n.Statement', 'lu_id': 1, 'lexeme': '선언서', 'fe_list': ['speaker', 'topic'], 'surface_forms': ['선언서를'], 'mapSejong': False, 'pos': 'NNG', 'frameName': 'Statement', 'lemma_var': ['선언서'], 'publish': True, 'ko_annotation_id': [1], 'sejong_annotation_id': [], 'en_lu': ['declaration']}
{'fid': 43, 'lu': '선언서.n.Statement', 'lu_id': 1, 'lexeme': '선언서', 'fe_list': ['speaker', 'topic'], 'surface_forms': ['선언서를'], 'mapSejong': False, 'pos': 'NNG', 'frameName': 'Statement', 'lemma_var': ['선언서'], 'publish': True, 'ko_annotation_id': [1], 'sejong_annotation_id': [], 'en_lu': ['declaration']}
